In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Fetch historical data for NASDAQ and NSE indices
start = '2010-01-01'
end = '2023-05-01'
nasdaq_data = yf.download('^IXIC', start=start, end=end).fillna(method='ffill')
nse_data = yf.download('^NSEI', start=start, end=end).fillna(method='ffill')

if len(nasdaq_data) != len(nse_data):
    min_length = min(len(nasdaq_data), len(nse_data))
    nasdaq_data = nasdaq_data[-min_length:]
    nse_data = nse_data[-min_length:]

correlation_coefficient = np.corrcoef(nasdaq_data['Close'], nse_data['Close'])[0, 1]

if correlation_coefficient > 0:
    relationship = 'positive'
elif correlation_coefficient < 0:
    relationship = 'negative'
else:
    relationship = 'no'

print(f"The correlation coefficient between NASDAQ and NSE indices is {correlation_coefficient:.2f}.")
print(f"The relationship between the two indices is {relationship}.")

# Calculate lead-lag relationship
lag_correlations = []
max_lag = 12

for lag in range(1, max_lag + 1):
    nasdaq_lagged = nasdaq_close.shift(lag)
    correlation = nasdaq_lagged.corr(nse_close)
    lag_correlations.append(correlation)

max_correlation = max(lag_correlations)
max_lag_index = lag_correlations.index(max_correlation) + 1

if max_lag_index < 0:
    relationship = "NASDAQ leads NSE by {} days".format(abs(max_lag_index))
elif max_lag_index > 0:
    relationship = "NSE leads NASDAQ by {} days".format(max_lag_index)
else:
    relationship = "No consistent lead-lag relationship"

print("Lead-Lag Relationship: ", relationship)

# Calculate Keltner Channel
def calculate_keltner_channel(data, n=20, atr_multiplier=2):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    atr = data['High'].rolling(n).max() - data['Low'].rolling(n).min()
    upper_band = typical_price + atr_multiplier * atr
    lower_band = typical_price - atr_multiplier * atr
    return upper_band, lower_band

nasdaq_data['Nasdaq Upper Band'], nasdaq_data['Nasdaq Lower Band'] = calculate_keltner_channel(nasdaq_data)
nse_data['NSE Upper Band'], nse_data['NSE Lower Band'] = calculate_keltner_channel(nse_data)

# Calculate Bollinger Bands
def calculate_bollinger_bands(data, n=20, std_multiplier=2):
    rolling_mean = data['Close'].rolling(n).mean()
    rolling_std = data['Close'].rolling(n).std()
    upper_band = rolling_mean + std_multiplier * rolling_std
    lower_band = rolling_mean - std_multiplier * rolling_std
    return upper_band, lower_band

nasdaq_data['Nasdaq Bollinger Upper'], nasdaq_data['Nasdaq Bollinger Lower'] = calculate_bollinger_bands(nasdaq_data)
nse_data['NSE Bollinger Upper'], nse_data['NSE Bollinger Lower'] = calculate_bollinger_bands(nse_data)

# Calculate MACD
def calculate_macd(data, n_fast=12, n_slow=26, n_signal=9):
    ema_fast = data['Close'].ewm(span=n_fast, min_periods=n_fast).mean()
    ema_slow = data['Close'].ewm(span=n_slow, min_periods=n_slow).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=n_signal, min_periods=n_signal).mean()
    macd_histogram = macd_line - signal_line
    return macd_line, signal_line, macd_histogram

nasdaq_data['Nasdaq MACD Line'], nasdaq_data['Nasdaq Signal Line'], nasdaq_data['Nasdaq MACD Histogram'] = calculate_macd(nasdaq_data)
nse_data['NSE MACD Line'], nse_data['NSE Signal Line'], nse_data['NSE MACD Histogram'] = calculate_macd(nse_data)

# Parameter Optimization
optimization_index = "NASDAQ"
if optimization_index == "NASDAQ":
    optimized_parameters = {
        'bollinger_period': 20,
        'bollinger_std_dev': 2,
        'keltner_period': 20,
        'keltner_multiplier': 1.5,
        'macd_fast_period': 12,
        'macd_slow_period': 26,
        'macd_signal_period': 9
    }
else:
    optimized_parameters = {
        'bollinger_period': 20,
        'bollinger_std_dev': 2,
        'keltner_period': 20,
        'keltner_multiplier': 2,
        'macd_fast_period': 12,
        'macd_slow_period': 26,
        'macd_signal_period': 9
    }